#  Understanding timestamps and BAT in CRACO data

Just want to understand what happens when we start CRACO and what the various times mean

With the `savescan` script we initiated a CRACO download. Using `ccaphdr --type timestamp` and looking at the log we get the following info
```
(venv) ban115@seren-01:20230322204016$ pwd
/data/big/craco/SB049014/scans/00/20230322204016
(venv) ban115@seren-01:20230322204016$ ccaphdr  --type timestamp *b02*c01*.fits 2>/dev/null
filename len syncbat npkt frame_id bat
ccap_b02_c01+f1.fits 1280000 0x126cd11ed694be 49014/0 VAST_1724-31 60025.86209525183 30 1 1 (2, 465, 1, 2) 1280000 731164704 0 937648558128 0xda50359030 0 32
ccap_b02_c01+f2.fits 1280000 0x126cd11ed694be 49014/0 VAST_1724-31 60025.86209525183 30 1 1 (2, 465, 1, 2) 1280000 731164704 0 937648558128 0xda50359030 0 32
ccap_b02_c01+f3.fits 1280000 0x126cd11ed694be 49014/0 VAST_1724-31 60025.86209525183 30 1 1 (2, 465, 1, 2) 1280000 731164704 0 937648558128 0xda50359030 0 32
ccap_b02_c01+f4.fits 1280000 0x126cd11ed694be 49014/0 VAST_1724-31 60025.86209525183 30 1 1 (2, 465, 1, 2) 1280000 731164704 0 937648558128 0xda50359030 0 32
ccap_b02_c01+f5.fits 1280000 0x126cd11ed694be 49014/0 VAST_1724-31 60025.86209525183 30 1 1 (2, 465, 1, 2) 1280000 731164704 0 937648558128 0xda50359030 0 32
ccap_b02_c01+f6.fits 1280000 0x126cd11ed694be 49014/0 VAST_1724-31 60025.86209525183 30 1 1 (2, 465, 1, 2) 1280000 731164704 0 937648558128 0xda50359030 0 32
(venv) ban115@seren-01:20230322204016$ grep 'Start bat' run.log 
INFO:craco.cardcap:Start bat is 0x126cda5034b800
(venv) ban115@seren-01:20230322204016$ ccaphdr ccap_b02_c02+f1.fits --type fits | grep SYNCBAT
ccap_b02_c02+f1.fits SYNCBAT = '0x126cd11ed694be'   / Hexadecimal BAT when frame ID was set to 0
```

In [22]:
import numpy as np

sync_bat = 0x126cd11ed694be  # BAT of "sync reset" event. Retrieved from PV
craco_go_bat = 0x126cda5034b800   # Bat of CRACO Go event. Retrieved from PV
fid0 = 731164704 # Frame ID from debug header of first data in CRACO download
bat0 = 0xda50359030 # bat from debug header of first data in CRACO download

OK so first thing to know is that BAT is in microseconds, but FRAMIDs (FIDS) are in ... frames .. which in this case is fine frequency samples, i.e. with the coarse sample rate 32/27 x 1 MHz, and an FFT of 64 for the fine frequuency channel, i.e. there are 54 microseconds sample, i.e. 54 microseconds per FID

In [15]:
fid_usec = int(27/32 * 64)
fid_usec

54

In [16]:
bfframe_samp = 2048 # number of fine samples per beamformer frame

All good so far. So there are a few things we'd like to check:
- CRACO Go event is set for middle of a beamformer frame
- fid0 and bat0 in the same debug header agree
- First debug header after craco go  event has expected FID and BAT


## Was the CRACO GO  event set for the middle of a beamformer frame?

In [17]:
# FRAMEID of CRACO go event
craco_go_fid = (craco_go_bat - sync_bat) / fid_usec
craco_go_fid

731165901.074074

In [19]:
# beamformer frame of CRACO GO aevent
craco_go_bfframe = craco_go_fid / bfframe_samp
craco_go_bfframe

357014.6001338252

In [21]:
craco_go_fid % bfframe_samp

1229.0740740299225

OK - so we can see that the CRACO go event was set for a BAT what was 1229 samples, or 0.6 beamformer frames, - roughtly the middle, of beamformer frame 357014. Fair enough.
On those grounds we'd expect CRACO to start on the next beamformer frame boundary.

In [26]:
expected_fid0 = int(np.ceil(craco_go_bfframe + 1)*bfframe_samp)
expected_fid0

731168768

## Does FID0 and BAT0 in the first debug header agree on the same time?

In principle the frame ID = 0 when BAT = syncbat, and everything increases linearly therafter. So we should be able to count frames since SYNCBAT and get the same BAT as the debug header. So let's see if that works.

In [28]:
expected_bat0 = sync_bat + fid_usec*fid0
expected_bat0

5186234485029758

In [29]:
hex(expected_bat0)

'0x126cda5033bb7e'

In [30]:
hex(bat0)

'0xda50359030'

OOPS! bat0 is only the first 40 bits of bat - let's mask in the rest from, say, syncbat

In [32]:
bat0full = bat0 | (sync_bat & (0xffffffff << 40))
hex(bat0full)

'0x126cda50359030'

OK that loks right, so now we'd expect expect bat0full and expected_bat0 to agree

In [33]:
expected_bat0 - bat0full

-119986

Oops, but they don't?!! Somehow we've lost 119986 microseconds. bat0 is slow by that amount. But is it bat0? or fid0 that's to blame?

## - Does the first debug header after craco go  event has expected FID and BAT?

Hmm, we know either FID0 or BAT0 is somehow incorrect. But which is wrong? And why?
Recall from earlier know when the CRACO go BAT was sent, and the first CRACO data should be the first beamformer frame after that bat. So let's look at that.

In [36]:
craco_go_bfframe # beamformer frame number when CRACO GO event was set

357014.6001338252

In [37]:
expected_fid0 # First FID in first whole beamformer frame after CRACO go event

731168768

In [41]:
fid0 - expected_fid0

-4064

WHOOOPS! Somehow the first data we got had an FID that 4068 frames *before* the CRACO Go event! That's 4096 - 32 frames early?! How about the BATS?

In [50]:
bat0full - craco_go_bat

55344

OK - so the first BAT we got from the CRACO data was 55344 usec after the CRACO Go event. This makse sense.

# Conclustion
It looks like frameID and BATs don't agree in the CRACO data. It looks like the frame IDs are about 4096 samples earlier than the BATS.